# Reading file

In [1]:
import pandas as pd
# I can give a number or use None to remove maximum ceiling & display all columns
pd.options.display.max_columns = None


# Setting up GPT 

In [3]:
from langchain.chat_models import ChatOpenAI

# Use temperature=0 to get the same results every time
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key = openai_api_key)


# Following WaPo example

## Create training data in excel
I filtered the data by those mentioning emails/ correspondence/ text messages and mostly trained on that but also included some empty ones

In [4]:
# Read Document
df_public_records_for_emails=pd.read_csv("save_df_public_records_25102023.csv")
df_public_records_for_emails

,Unnamed: 0,pr_req_url,duplicate_pr_req_url,uploaded_dates,extracted_received_date,received_via,first_file_uploaded_date,contact,request_opened_date,request_opened_time,request_opened_by,last_file_uploaded_date,duration_from_received_to_first_uploaded_date,duration_from_received_to_last_uploaded_date,request_opened_datetime,duration_from_open_to_last_uploaded_date,status_scraped,full_request_message,is_responder_requesting_email,request_closed_content_text,exploded_dep_names,num_of_departments,original_dep_list,exploded_dep_renamed,req_id,req_url,req_closed_time,duration_from_received_to_closed,duration_from_received_to_closed_among_closed_requests,exploded_dep_renamed_saving,frequency_of_closure_text,extended_category_request_closed_content_text,category_request_closed_content_text
0,0,https://miami.nextrequest.com/requests/18-1,NaN,[],2018-01-01,web,NaN,Ryan Steed,2018-01-01,7:15 AM,the requester,NaN,NaN,NaN,2018-01-01 7:15:00,NaN,Closed,This should be directed to:\nRyan Steed\nThe f...,yes,Previous Request was reopened. This Request wa...,No departments assigned,1,['No departments assigned'],No departments assigned,18-1,https://miami.nextrequest.com/client/requests/...,2018-01-02 10:13:00,1 days 10:13:00,1 days 10:13:00,No departments assigned,1.0,duplicate,referred_or_fowarded_or_duplicate
1,1,https://miami.nextrequest.com/requests/18-10,First Duplicate,[],2018-01-02,web,NaN,Ryan Steed,2018-01-02,3:28 PM,the requester,NaN,NaN,NaN,2018-01-02 15:28:00,NaN,Closed,We are requesting all police reports for any i...,no,Request was forwarded to Police,No departments assigned,1,['No departments assigned'],No departments assigned,18-10,https://miami.nextrequest.com/client/requests/...,2018-01-03 9:46:00,1 days 09:46:00,1 days 09:46:00,No departments assigned,4.0,referred_to_police,referred_or_fowarded_or_duplicate
2,2,https://miami.nextrequest.com/requests/18-100,First Duplicate,[],2018-01-24,email,NaN,Michelle Redruello,2018-01-24,10:00 AM,Staff,NaN,NaN,NaN,2018-01-24 10:00:00,NaN,Closed,"Dear Chief Jorge: \n \nAs you know, this firm ...",yes,FulfilledAll responsive and non-exempt public ...,No departments assigned,1,['No departments assigned'],No departments assigned,18-100,https://miami.nextrequest.com/client/requests/...,2018-01-24 10:01:00,0 days 10:01:00,0 days 10:01:00,No departments assigned,376.0,documents_provided_download,fulfilled
3,3,https://miami.nextrequest.com/requests/18-1000,First Duplicate,[],2018-06-28,email,NaN,Stephanie Schloss-Sassi,2018-06-28,11:02 AM,Staff,NaN,NaN,NaN,2018-06-28 11:02:00,NaN,Closed,I would also like to request any email that Ad...,yes,Closed at Direction/Approval of the RequestorT...,I.T.,1,['I.T.'],I.T.,18-1000,https://miami.nextrequest.com/client/requests/...,2018-06-28 11:03:00,0 days 11:03:00,0 days 11:03:00,I.T.,549.0,Closed_at_Direction_or_Approval_of_the_Requestor,closed_at_approval_of_requestor
4,4,https://miami.nextrequest.com/requests/18-1001,NaN,['Uploaded: 12/18/2018'],2018-06-28,email,2018-12-18,Ryan Steed,2018-06-28,11:03 AM,Staff,2018-12-18,173 days,173 days,2018-06-28 11:03:00,172 days 12:57:00,Closed,"Please, I would like to make a public records ...",no,The City does not have any responsive document...,Building,1,['Building'],Building,18-1001,https://miami.nextrequest.com/client/requests/...,2018-12-18 9:28:00,173 days 09:28:00,173 days 09:28:00,Building,2854.0,no_responsive_documents_available,no_responsive_documents_available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24533,24533,https://miami.nextrequest.com/requests/23-995,NaN,[],2023-03-16,web,NaN,Michael Mesa,2023-03-16,2:39 PM,the requester,NaN,NaN,NaN,2023-03-16 14:39:00,NaN,Closed,Property: Core Link At Douglas Address:3060 SW...,NaN,The City does not have any responsive document...,Fire - Property Search,1,['Fire - Property Search'],Fire - Property Search,23-995,https://miami.nextrequest.com/client/requests/...,2023-03-17 

# Filter by cases where training data is filled

In [5]:
known_df_public_records_for_emails = df_public_records_for_emails[df_public_records_for_emails.is_responder_requesting_email.notna()].copy()
unknown_df_public_records_for_emails = df_public_records_for_emails[df_public_records_for_emails.is_responder_requesting_email.isna()].copy()

In [6]:
known_df_public_records_for_emails.shape


(487, 33)

In [7]:
known_df_public_records_for_emails[known_df_public_records_for_emails.is_responder_requesting_email.isna()]

,Unnamed: 0,pr_req_url,duplicate_pr_req_url,uploaded_dates,extracted_received_date,received_via,first_file_uploaded_date,contact,request_opened_date,request_opened_time,request_opened_by,last_file_uploaded_date,duration_from_received_to_first_uploaded_date,duration_from_received_to_last_uploaded_date,request_opened_datetime,duration_from_open_to_last_uploaded_date,status_scraped,full_request_message,is_responder_requesting_email,request_closed_content_text,exploded_dep_names,num_of_departments,original_dep_list,exploded_dep_renamed,req_id,req_url,req_closed_time,duration_from_received_to_closed,duration_from_received_to_closed_among_closed_requests,exploded_dep_renamed_saving,frequency_of_closure_text,extended_category_request_closed_content_text,category_request_closed_content_text


In [8]:
# nan_after_mapping = y_train.isna()
# rows_with_nan = known_df_public_records_for_emails[nan_after_mapping]
# known_df_public_records_for_emails['is_responder_requesting_email']


# Using Training

In [9]:
known_df_public_records_for_emails['full_request_message'].fillna('filling missing_values for training model to function', inplace=True)
unknown_df_public_records_for_emails['full_request_message'].fillna('filling missing_values for training model to function', inplace=True)


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

# with 'full_request_message' as a column of text data and 
# 'is_responder_requesting_email' as the target variable with values 'yes' or 'no'.

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(known_df_public_records_for_emails['full_request_message'])

# Map target labels to binary values
y = known_df_public_records_for_emails['is_responder_requesting_email'].map({'yes': 1, 'no': 0})

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Logistic Regression classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Predict on the validation set
y_pred = clf.predict(X_val)

# Evaluate the classifier's performance
print("Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred))


Accuracy: 0.8673469387755102

Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.95      0.87        44
           1       0.96      0.80      0.87        54

    accuracy                           0.87        98
   macro avg       0.87      0.88      0.87        98
weighted avg       0.88      0.87      0.87        98



In [11]:
unknown_df_public_records_for_emails.to_csv("first_trained_unknown_df_public_records_for_emails.csv")